In [1]:
!pip install gurobipy

In [2]:
#from google.colab import files
#uploaded = files.upload()  # This will prompt you to select and upload a file.

In [3]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [4]:
def castRisk(character):
  mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'F': 6}
  if character in mapping.keys():
    return mapping[character]
  #else:
  #  print(character)
  #  return 1

In [5]:
def distinct(d, var, num):
    modVar = []
    for index in range(num):
        if d[index]:
            modVar.append(var[index])
    occurence = {}
    for item in modVar:
        if item not in occurence.keys():
            occurence[item] = 1
        else:
            occurence[item] = occurence[item] + 1
    return len(occurence.keys())

In [6]:
df = pd.read_excel('/Users/michael_khalfin/Downloads/Book6.xlsx')
df = df.head(33)

In [7]:
df.head()

,Project Title,Location,Project Developer,Type,Mechanism,Vintage,Quantity,Dollar,Registry,Standard,Grade
0,Promoting hydroelectric green energy in India,India,EVI Green Markets Pte. Ltd,Hydropower,Avoidance,2019.0,2617.0,2.6300,CDM,CDM,B
1,Promoting hydroelectric green energy in India,India,EVI Green Markets Pte. Ltd,Hydropower,Avoidance,2018.0,20000.0,2.6300,CDM,CDM,B
2,Promoting hydroelectric green energy in India,India,EVI Green Markets Pte. Ltd,Hydropower,Avoidance,2017.0,20000.0,2.6300,CDM,CDM,B
3,The Nut Collectors | Guardians of the Forest,Peru,Bosques Amazónicos,REDD+,Avoidance,2020.0,29500.0,11.2564,Verra,VCS,F
4,The Nut Collectors | Guardians of the Forest,Peru,Bosques Amazónicos,REDD+,Avoidance,2018.0,30000.0,10.0152,Verra,VCS,F


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Project Title      33 non-null     object 
 1   Location           33 non-null     object 
 2   Project Developer  33 non-null     object 
 3   Type               33 non-null     object 
 4   Mechanism          33 non-null     object 
 5   Vintage            33 non-null     float64
 6   Quantity           33 non-null     float64
 7   Dollar             33 non-null     float64
 8   Registry           33 non-null     object 
 9   Standard           33 non-null     object 
 10  Grade              33 non-null     object 
dtypes: float64(3), object(8)
memory usage: 3.0+ KB


In [9]:
# Initialize model
model = gp.Model("MIP_Model")

Restricted license - for non-production use only - expires 2023-10-25


In [10]:
# Number of projects in the data table
numProjDev = df.shape[0]

In [11]:
# Add variables to the model
x = model.addVars(numProjDev, vtype=GRB.INTEGER, name="quantity")
d = model.addVars(numProjDev, vtype=GRB.BINARY, name="devTF")
#g = model.addVars(numProjDev, vtype=GRB.INTEGER, name="registry")
#v = model.addVars(numProjDev, vtype=GRB.INTEGER, name="vintage")
#t = model.addVars(numProjDev, vtype=GRB.INTEGER, name="typeProj")
#l = model.addVars(numProjDev, vtype=GRB.INTEGER, name="location")
#ch = model.addVars(numProjDev, vtype=GRB.INTEGER, name="mechanism")

In [12]:
df['Grade'].values

array(['B', 'B', 'B', 'F', 'F', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'F',
       'F', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'C', 'F', 'F', 'D', 'B', 'C', 'B'], dtype=object)

In [13]:
obj = sum(elem1 * castRisk(elem2) for elem1, elem2 in zip(x, df['Grade']))
model.setObjective(obj, GRB.MINIMIZE)

In [14]:
df['Dollar'][1]

2.63

In [15]:
model.addConstr(gp.quicksum(df['Dollar'][i]*x[i] for i in range(numProjDev)) <= 1000, "price")
for i in range(numProjDev):
  model.addConstr(x[i] <= df["Quantity"][i], "quantity")
  model.addConstr(d[i] * x[i] - x[i] >= -0.5, "1st binary constraint")
  model.addConstr(d[i] * x[i] - d[i] >= -0.5, "2nd binary constraint")
model.addConstr(gp.quicksum(d[i] for i in range(numProjDev)) >= 20)

<gurobi.Constr *Awaiting Model Update*>

In [16]:
model.addConstr(distinct(d, df['Vintage'], numProjDev) >= 1, "vintage")
model.addConstr(distinct(d, df['Registry'], numProjDev) >= 1, "registry")
model.addConstr(distinct(d, df['Location'], numProjDev) >= 1, "location")
model.addConstr(distinct(d, df['Mechanism'], numProjDev) >= 1, "mechanism")
model.addConstr(distinct(d, df['Type'], numProjDev) >= 1, "typeProject")
model.addConstr(gp.quicksum(x[i] for i in range(numProjDev)) >= 100, "quota")

<gurobi.Constr *Awaiting Model Update*>

In [17]:
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 41 rows, 66 columns and 132 nonzeros
Model fingerprint: 0x1b3f5d22
Model has 66 quadratic constraints
Variable types: 0 continuous, 66 integer (33 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
  QRHS range       [5e-01, 5e-01]
Presolve added 20 rows and 0 columns
Presolve removed 0 rows and 7 columns
Presolve time: 0.01s
Presolved: 61 rows, 59 columns, 206 nonzeros
Variable types: 0 continuous, 59 integer (30 binary)
Found heuristic solution: objective 1457.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 1457 

Optimal solution found (toler

In [18]:
for v in model.getVars():
    print(f"{v.varName} = {v.x}")

quantity[0] = -0.0
quantity[1] = -0.0
quantity[2] = -0.0
quantity[3] = -0.0
quantity[4] = -0.0
quantity[5] = -0.0
quantity[6] = -0.0
quantity[7] = -0.0
quantity[8] = -0.0
quantity[9] = 1.0
quantity[10] = 1.0
quantity[11] = 1.0
quantity[12] = 1.0
quantity[13] = 1.0
quantity[14] = -0.0
quantity[15] = -0.0
quantity[16] = 0.0
quantity[17] = 0.0
quantity[18] = 1.0
quantity[19] = 1.0
quantity[20] = 1.0
quantity[21] = 1.0
quantity[22] = 1.0
quantity[23] = 1.0
quantity[24] = 1.0
quantity[25] = 1.0
quantity[26] = 1.0
quantity[27] = 1.0
quantity[28] = 1.0
quantity[29] = 1.0
quantity[30] = 1.0
quantity[31] = 81.0
quantity[32] = 1.0
devTF[0] = -0.0
devTF[1] = -0.0
devTF[2] = -0.0
devTF[3] = -0.0
devTF[4] = -0.0
devTF[5] = -0.0
devTF[6] = -0.0
devTF[7] = -0.0
devTF[8] = -0.0
devTF[9] = 1.0
devTF[10] = 1.0
devTF[11] = 1.0
devTF[12] = 1.0
devTF[13] = 1.0
devTF[14] = 0.0
devTF[15] = 0.0
devTF[16] = 0.0
devTF[17] = 0.0
devTF[18] = 1.0
devTF[19] = 1.0
devTF[20] = 1.0
devTF[21] = 1.0
devTF[22] = 1.0
devT